## 🔹 Partie 1 : Introduction Théorique

### Qu'est-ce que l'Alignement (RLHF) ?

L'alignement vise à corriger les comportements indésirables du modèle (hallucinations, toxicité, réponses verbeuses) qui persistent même après le SFT.

### Le Pipeline RLHF en 3 Étapes

| Étape | Nom | Description | Objectif |
|-------|-----|-------------|----------|
| **1** | **Collecte de Préférences** | On génère plusieurs réponses pour un prompt et on demande (à un humain ou une IA) de choisir la meilleure. | Créer un dataset `(Prompt, Chosen, Rejected)` |
| **2** | **Reward Model (RM)** | On entraîne un modèle à prédire un score de qualité (scalaire) pour une réponse donnée. | Apprendre à distinguer le "bon" du "mauvais" |
| **3** | **PPO (RL Loop)** | On utilise le RM pour guider le modèle de langage via l'apprentissage par renforcement. | Maximiser le score de récompense tout en restant cohérent |

### Schéma du Processus

```mermaid
graph LR
    A[Modèle SFT] --> B[Génération de Réponses]
    B --> C{Préférence Humaine}
    C -->|Gagnant| D[Chosen]
    C -->|Perdant| E[Rejected]
    D & E --> F[Entraînement Reward Model]
    F --> G[Boucle PPO]
    G --> H[Modèle Aligné (InstructGPT)]
```

## 🔹 Partie 2 : Configuration & Imports

In [ ]:
# %% Cell 1: Imports et Configuration

# ============================================================================
# IMPORTS DES BIBLIOTHÈQUES NÉCESSAIRES
# ============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import math
import os
import json
import random
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import dataclass
from transformers import AutoTokenizer

# ============================================================================
# CONFIGURATION POUR LA REPRODUCTIBILITÉ
# ============================================================================
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# ============================================================================
# DÉTECTION DU DEVICE
# ============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device utilisé : {device}")

# ============================================================================
# CHEMINS DES FICHIERS
# ============================================================================
SFT_MODEL_PATH = "models/post_training/model_sft_FINAL.pt"
TOKENIZER_PATH = "models/post_training/tokenizer/"

print(f"📂 Modèle SFT cible : {SFT_MODEL_PATH}")

## 🔹 Partie 3 : Chargement du Modèle SFT

Nous devons recharger l'architecture exacte utilisée lors du Post-Training (`TinyDecoderLM`).

In [ ]:
# %% Cell 2: Définition de l'Architecture (Identique au Post-Training)

# ============================================================================
# CONFIGURATION DU MODÈLE
# ============================================================================
@dataclass
class ModelConfig:
    vocab_size: int = 50260 # Ajusté pour inclure les tokens spéciaux
    d_model: int = 512
    n_heads: int = 8
    n_layers: int = 8
    d_ff: int = 2048
    block_size: int = 256

# ============================================================================
# MODULES DU TRANSFORMER
# ============================================================================
class CausalSelfAttention(torch.nn.Module):
    def __init__(self, d_model, n_heads, block_size):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads

        self.qkv = torch.nn.Linear(d_model, 3 * d_model)
        self.proj = torch.nn.Linear(d_model, d_model)

        mask = torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size)
        self.register_buffer("mask", mask)

    def forward(self, x):
        B, T, C = x.shape
        qkv = self.qkv(x)
        q, k, v = qkv.split(C, dim=2)
        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float("-inf"))
        att = torch.nn.functional.softmax(att, dim=-1)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        return self.proj(y)

class Block(torch.nn.Module):
    def __init__(self, d_model, n_heads, d_ff, block_size):
        super().__init__()
        self.ln1 = torch.nn.LayerNorm(d_model)
        self.attn = CausalSelfAttention(d_model, n_heads, block_size)
        self.ln2 = torch.nn.LayerNorm(d_model)
        self.ff = torch.nn.Sequential(
            torch.nn.Linear(d_model, d_ff),
            torch.nn.GELU(),
            torch.nn.Linear(d_ff, d_model),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class TinyDecoderLM(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.tok_emb = torch.nn.Embedding(cfg.vocab_size, cfg.d_model)
        self.pos_emb = torch.nn.Embedding(cfg.block_size, cfg.d_model)
        self.blocks = torch.nn.ModuleList([
            Block(cfg.d_model, cfg.n_heads, cfg.d_ff, cfg.block_size)
            for _ in range(cfg.n_layers)
        ])
        self.ln_f = torch.nn.LayerNorm(cfg.d_model)
        self.head = torch.nn.Linear(cfg.d_model, cfg.vocab_size, bias=False)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)
        x = self.tok_emb(idx) + self.pos_emb(pos)
        
        for blk in self.blocks:
            x = blk(x)
        
        x = self.ln_f(x)
        logits = self.head(x)
        
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, self.cfg.vocab_size), targets.view(-1), ignore_index=-100)
        
        return logits, loss

print("✅ Architecture TinyDecoderLM définie.")

In [ ]:
# %% Cell 3: Chargement du Tokenizer et du Modèle

print("📥 Chargement du Tokenizer et du Modèle SFT...")

# 1. Charger le Tokenizer
if os.path.exists(TOKENIZER_PATH):
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
    print(f"✅ Tokenizer chargé depuis {TOKENIZER_PATH}")
else:
    print("⚠️ Tokenizer non trouvé, chargement par défaut (gpt-neox-20b)")
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
    tokenizer.pad_token = tokenizer.eos_token
    # Ajouter les tokens spéciaux si nécessaire
    special_tokens = {'additional_special_tokens': ['<instruction>', '<reasoning>', '<answer>']}
    tokenizer.add_special_tokens(special_tokens)

vocab_size = len(tokenizer)
print(f"📚 Taille du vocabulaire : {vocab_size}")

# 2. Charger le Modèle SFT
config = ModelConfig(vocab_size=vocab_size)
model_sft = TinyDecoderLM(config).to(device)

if os.path.exists(SFT_MODEL_PATH):
    checkpoint = torch.load(SFT_MODEL_PATH, map_location=device)
    # Gérer les différences de clés si nécessaire (prefixe 'module.' etc)
    state_dict = checkpoint['model_state_dict']
    model_sft.load_state_dict(state_dict)
    print(f"✅ Modèle SFT chargé depuis {SFT_MODEL_PATH}")
else:
    print(f"⚠️ Fichier {SFT_MODEL_PATH} introuvable. Initialisation aléatoire pour démo.")

model_sft.eval()

## 🔹 Étape 1 : Collecte de Données de Préférence (Good/Bad)

Nous simulons un dataset de préférences où pour chaque prompt, nous avons une réponse "choisie" (Chosen) et une réponse "rejetée" (Rejected).

In [ ]:
# %% Cell 4: Création du Dataset de Préférences

# Simulation de données (Prompt, Chosen, Rejected)
# Dans la réalité, ces données proviennent d'annotations humaines
preference_data = [
    {
        "prompt": "<instruction> Write a function to add two numbers <reasoning>",
        "chosen": "def add(a, b):\n    return a + b",
        "rejected": "def add(a, b):\n    print('Adding')\n    return a + b"
    },
    {
        "prompt": "<instruction> Create a list of squares <reasoning>",
        "chosen": "[x**2 for x in range(10)]",
        "rejected": "l = []\nfor i in range(10):\n    l.append(i*i)"
    },
    {
        "prompt": "<instruction> Check if even <reasoning>",
        "chosen": "def is_even(n): return n % 2 == 0",
        "rejected": "def is_even(n):\n    if n % 2 == 0:\n        return True\n    else:\n        return False"
    },
    {
        "prompt": "<instruction> Import pandas <reasoning>",
        "chosen": "import pandas as pd",
        "rejected": "import pandas"
    }
] * 10 # Dupliquer pour avoir un peu de volume

print(f"📝 Dataset de préférences créé : {len(preference_data)} exemples")
print(f"Exemple 1 Chosen: {preference_data[0]['chosen']}")
print(f"Exemple 1 Rejected: {preference_data[0]['rejected']}")

## 🔹 Étape 2 : Reward Model (RM)

Le Reward Model est un modèle qui prend une séquence en entrée et retourne un **score scalaire**.
Nous adaptons notre `TinyDecoderLM` en remplaçant la tête de vocabulaire par une tête de régression.

In [ ]:
# %% Cell 5: Définition et Entraînement du Reward Model

class RewardModel(nn.Module):
    def __init__(self, base_model, d_model):
        super().__init__()
        # On réutilise les couches du modèle de base (Transfer Learning)
        self.tok_emb = base_model.tok_emb
        self.pos_emb = base_model.pos_emb
        self.blocks = base_model.blocks
        self.ln_f = base_model.ln_f
        
        # Nouvelle tête de score (projection vers 1 scalaire)
        self.score_head = nn.Linear(d_model, 1, bias=False)

    def forward(self, idx):
        B, T = idx.shape
        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)
        x = self.tok_emb(idx) + self.pos_emb(pos)
        
        for blk in self.blocks:
            x = blk(x)
            
        x = self.ln_f(x)
        
        # On utilise le dernier token pour prédire le score de la séquence entière
        # (Padding token handling serait nécessaire dans un cas réel complexe)
        last_token_hidden = x[:, -1, :]
        score = self.score_head(last_token_hidden)
        return score

print("🏗️ Construction du Reward Model...")
# Initialiser le RM avec les poids du SFT
reward_model = RewardModel(model_sft, config.d_model).to(device)

# Optimiseur pour le RM
optimizer_rm = torch.optim.AdamW(reward_model.parameters(), lr=1e-5)

print("🏋️‍♂️ Entraînement du Reward Model (Bradley-Terry Loss)...")
rm_losses = []

for epoch in range(3): # Quelques époques rapides
    total_loss = 0
    for ex in preference_data:
        # Préparer les inputs
        # Format: Prompt + Réponse
        text_chosen = ex['prompt'] + " " + ex['chosen']
        text_rejected = ex['prompt'] + " " + ex['rejected']
        
        idx_chosen = torch.tensor([tokenizer.encode(text_chosen)], device=device)
        idx_rejected = torch.tensor([tokenizer.encode(text_rejected)], device=device)
        
        # Forward
        r_chosen = reward_model(idx_chosen)
        r_rejected = reward_model(idx_rejected)
        
        # Loss: -log(sigmoid(r_chosen - r_rejected))
        loss = -torch.log(torch.sigmoid(r_chosen - r_rejected))
        
        optimizer_rm.zero_grad()
        loss.backward()
        optimizer_rm.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(preference_data)
    rm_losses.append(avg_loss)
    print(f"   Epoch {epoch+1}: Loss = {avg_loss:.4f}")

print("✅ Reward Model entraîné.")

## 🔹 Étape 3 : Boucle RLHF (PPO Simplifié)

Nous utilisons maintenant le Reward Model pour guider la génération.
Pour simplifier cette démo, nous implémentons une boucle **Policy Gradient** simple avec pénalité KL.

**Objectif** : Maximiser $Reward(x, y) - \beta \cdot KL(\pi_{\theta} || \pi_{ref})$

In [ ]:
# %% Cell 6: Boucle RLHF

# 1. Modèle de Référence (Ref Model) - Figé
# Sert à calculer la divergence KL pour éviter que le modèle ne s'éloigne trop
model_ref = TinyDecoderLM(config).to(device)
model_ref.load_state_dict(model_sft.state_dict())
model_ref.eval()
for p in model_ref.parameters():
    p.requires_grad = False

# 2. Optimiseur pour la Policy (Modèle SFT qu'on aligne)
optimizer_ppo = torch.optim.AdamW(model_sft.parameters(), lr=1e-6)

# Hyperparamètres RL
BETA = 0.1 # Coefficient de pénalité KL
STEPS = 50

print("🚀 Démarrage de la boucle RLHF...")

prompts_rl = [
    "<instruction> Write a python function <reasoning>",
    "<instruction> Import math library <reasoning>",
    "<instruction> Define a class <reasoning>"
]

history_rewards = []

for step in range(STEPS):
    # A. Rollout (Génération)
    prompt = random.choice(prompts_rl)
    idx = torch.tensor([tokenizer.encode(prompt)], device=device)
    
    # On génère un seul token pour simplifier l'exemple PPO step
    # Dans la réalité, on génère une séquence entière
    with torch.no_grad():
        logits, _ = model_sft(idx)
        probs = F.softmax(logits[:, -1, :], dim=-1)
        action = torch.multinomial(probs, 1) # Token choisi
    
    # Séquence complète
    idx_new = torch.cat((idx, action), dim=1)
    
    # B. Évaluation (Reward)
    reward = reward_model(idx_new).detach()
    
    # C. Calcul KL Divergence
    with torch.no_grad():
        logits_ref, _ = model_ref(idx)
        probs_ref = F.softmax(logits_ref[:, -1, :], dim=-1)
        prob_ref_token = probs_ref.gather(1, action)
    
    # Probabilité du token sous la policy actuelle
    # On doit refaire un forward avec grad activé
    logits_pol, _ = model_sft(idx)
    probs_pol = F.softmax(logits_pol[:, -1, :], dim=-1)
    prob_pol_token = probs_pol.gather(1, action)
    
    # KL approx: log(p_pol) - log(p_ref)
    kl = torch.log(prob_pol_token) - torch.log(prob_ref_token)
    
    # Reward Total
    total_reward = reward - BETA * kl
    
    # D. Update (Policy Gradient)
    # Loss = -log(p) * R
    loss = -torch.log(prob_pol_token) * total_reward
    
    optimizer_ppo.zero_grad()
    loss.backward()
    optimizer_ppo.step()
    
    history_rewards.append(total_reward.item())
    
    if step % 10 == 0:
        print(f"   Step {step}: Reward = {reward.item():.4f}, Total (w/ KL) = {total_reward.item():.4f}")

plt.plot(history_rewards)
plt.title("Évolution du Reward (RLHF)")
plt.xlabel("Step")
plt.ylabel("Total Reward")
plt.show()

## 🔹 Comparaison Finale

Comparons la génération avant et après alignement.

In [ ]:
# %% Cell 7: Test de Génération

def generate(model, prompt):
    model.eval()
    idx = torch.tensor([tokenizer.encode(prompt)], device=device)
    with torch.no_grad():
        # Génération simple
        for _ in range(20):
            logits, _ = model(idx)
            idx_next = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)
            idx = torch.cat((idx, idx_next), dim=1)
    return tokenizer.decode(idx[0].tolist())

test_prompt = "<instruction> Write a function <reasoning>"

print("🤖 Modèle de Référence (SFT - Avant RLHF):")
print(generate(model_ref, test_prompt))
print("-" * 50)
print("✨ Modèle Aligné (Après RLHF):")
print(generate(model_sft, test_prompt))

## 🏁 Conclusion

Nous avons complété le pipeline :
1.  **Pre-Training** : Apprentissage de la syntaxe.
2.  **Post-Training** : Apprentissage des instructions.
3.  **Alignment** : Optimisation par récompense.

Le modèle est maintenant prêt à être déployé ! 🚀